### Import necessary libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install dash
!pip install dash-bootstrap-components

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import re
import gc
import pickle

import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import datetime

### TIME visualization

In [ ]:
visual_df = pd.read_excel(r'/content/drive/Shareddrives/General/Trainee Folders/Tuan Phan/NUTS3/csv_ForVisualize/time_visual.xlsx', index_col = 0)
visual_df

In [ ]:
#Convert Period and Available to numeric
visual_df['Period'] = pd.to_numeric(visual_df['Period'], errors='coerce')
visual_df['Available'] = pd.to_numeric(visual_df['Available'], errors='coerce')

In [ ]:
today = datetime.date.today()

cur_year = today.year

# Divide years into periods
first_90s = visual_df[(visual_df['Period'] < 1990)]
second_90s = visual_df[(visual_df['Period'] >= 1990) & (visual_df['Period'] < 2000)]

first_00s = visual_df[(visual_df['Period'] >= 2000) & (visual_df['Period'] <= 2010)]
second_00s = visual_df[(visual_df['Period'] >= 2011) & (visual_df['Period'] <= cur_year)]

period_future = visual_df[(visual_df['Period'] > cur_year)]

In [ ]:
df = visual_df
# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
                        html.H1('Period Availability Heatmap'),

                        # Dropdown for selecting period
                        dcc.Dropdown(
                            id='period-dropdown',
                            options=["1977 - 1989", "1990 - 1999", "2000 - 2010", f"2011 - {cur_year}", "Future"],
                            value=["first_90s", "second_90s", "first_00s", "second_00s", "Future"],
                            clearable=False
                        ),

                        # Graph component
                        dcc.Graph(id='heatmap', style={'height': '80vh'})
                        ])

# Callback to update the heatmap based on the selected period
@app.callback(
    Output('heatmap', 'figure'),
    [Input('period-dropdown', 'value')]
    )

def update_heatmap(selected_period):
  if selected_period == "1977 - 1989":
    filtered_df = first_90s
  elif selected_period == "1990 - 1999":
    filtered_df = second_90s
  elif selected_period == "2000 - 2010":
    filtered_df = first_00s
  elif selected_period == f"2011 - {cur_year}":
    filtered_df = second_00s
  else:
    filtered_df = period_future

  fig = px.imshow(
      filtered_df.pivot(index='table_id', columns='Period', values='Available'),
      labels=dict(x="Period", y="Table ID", color="Available"),
      x=filtered_df['Period'].unique(),
      y=filtered_df['table_id'].unique(),
      title=f"Availability Heatmap for {selected_period}"
      )

  fig.update_traces(colorscale=[[0, '#808080'], [1, '#008000']])

  fig.update_coloraxes(colorbar=dict(
      title="Available",
      titleside="right",
      tickmode="array",
      tickvals=[0, 1],
      ticktext=["No", "Yes"]
      ))

  return fig

# Run the app
if __name__ == '__main__':
  app.run_server(debug=True)

<IPython.core.display.Javascript object>

### COUNTRY visualization

In [4]:
today = datetime.date.today()
cur_year = today.year

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Period dropdown options and mapping
period_options = [
    {"label": "1977 - 1999", "value": "period_90s"},
    {"label": f"2000 - {cur_year}", "value": "period_00s"},
    {"label": "Future", "value": "Future"},
]

category_options = [
    "Agri-environmental indicators", "Annual national accounts", "Area", "Business demography",
    "Demography", "Energy", "Intellectual property rights", "Labour force survey", "National road",
    "Patent", "Police record", "Population and housing censuses 2001", "Population and housing censuses 2021",
    "Road Accident", "Structure of agricultural holdings", "Tourism"
]

with open("/content/drive/Shareddrives/General/Trainee Folders/Tuan Phan/NUTS3/csv_ForVisualize/geo_list", "rb") as fp:
   geo_l = pickle.load(fp)

geo_set = set()
for geo in geo_l:
  geo_set.add(geo[:2])

#Make a list contain country code (Only first 2 letters)
geo_options = list(geo_set)
geo_options.sort()

# Layout of the app
app.layout = dbc.Container(
    [
        html.H2("Geo Availability Heatmap", className="text-center mb-4"),
        dbc.Row([
            dbc.Col(
                html.Div([
                    html.Label("Select period", htmlFor="period-dropdown"),
                    dcc.Dropdown(
                        id="period-dropdown",
                        options=period_options,
                        value="first_90s",  # Default value
                        clearable=False
                    ),
                ]),
                width=4
            ),
            dbc.Col(
                html.Div([
                    html.Label("Select category", htmlFor="category-dropdown"),
                    dcc.Dropdown(
                        id="category-dropdown",
                        options=[{"label": cat, "value": cat} for cat in category_options],
                        value="Agri-environmental indicators",  # Default value
                        searchable=True,
                        clearable=False
                    ),
                ]),
                width=4
            ),
            dbc.Col(
                html.Div([
                    html.Label("Select geo", htmlFor="geo-dropdown"),
                    dcc.Dropdown(
                        id="geo-dropdown",
                        options=[{"label": cat, "value": cat} for cat in geo_options],
                        value="AL",  # Default value
                        searchable=True,
                        clearable=False
                    ),
                ]),
                width=4
            ),


        ]),
        dbc.Row([
            dbc.Col(dcc.Graph(id="heatmap"), width=12)
        ]),
    ],
    fluid=True
)

# Callback to update the heatmap based on selected period and category
@app.callback(
    Output("heatmap", "figure"),
    [Input("period-dropdown", "value"),
     Input("category-dropdown", "value"),
     Input("geo-dropdown", "value")]
)
def update_heatmap(selected_period, selected_categories, selected_geo):
    # Further filter based on selected categories
    # Apply filter for df to visualize
    df = pd.read_excel(f'/content/drive/Shareddrives/General/Trainee Folders/Tuan Phan/NUTS3/csv_ForVisualize/{selected_categories}.xlsx')

    #Select all the geo of target country
    df = df[df['geo'].str.startswith(selected_geo)]

    period_90s = df[(df['Period'] < 2000)]

    period_00s = df[(df['Period'] >= 2000) & (df['Period'] <= cur_year)]

    period_future = df[(df['Period'] > cur_year)]

    # Filter data based on selected period
    if selected_period == "period_90s":
        filtered_df = period_90s
    elif selected_period == "period_00s":
        filtered_df = period_00s
    else:
        filtered_df = period_future

    # Create the heatmap
    fig = px.imshow(
        filtered_df.pivot(index="geo", columns="Period", values="Available"),
        labels={"x": "Period", "y": "geo", "color": "Available"},
        x=filtered_df['Period'].unique(),
        y=filtered_df['geo'].unique(),
        title=f"Availability Heatmap for {selected_period} \n in category: {selected_categories} \n and country: {selected_geo}",
        color_continuous_scale=[[0, '#808080'], [1, '#00FF00']]
    )

    # Handle edge case: When the dataframe only contains `0`
    if filtered_df['Available'].nunique() == 1 and filtered_df['Available'].iloc[0] == 0:
        fig.update_traces(
            colorscale=[[0, '#808080'], [1, '#808080']],  # Use grey for all values when only 0 is present
            zmin=0,
            zmax=1
        )
    else:
        fig.update_traces(
            colorscale=[[0, '#808080'], [1, '#00FF00']],  # Use green values = 1, grey for 0
            zmin=0,
            zmax=1
        )

    # Customize colorbar
    fig.update_coloraxes(colorbar=dict(
        title="Available",
        titleside="right",
        tickmode="array",
        tickvals=[0, 1],
        ticktext=["No", "Yes"]
    ))


    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>